In [3]:
#Code for improved fMRI-bias learning
#OV Jan 2021
####IMPORTS
import tensorflow as tf
import socket
import neurite as ne
import neurite_sandbox as nes
import numpy as np
import keras
import freesurfer as fs
import scipy
from   skimage.measure import label, regionprops
from keras.optimizers import Adam
from keras import backend as K
from keras import layers as KL
from nibabel import freesurfer as nbfs
#from freesurfer import deeplearn as fsd
#OV: This imports all the local settings, e.g. subjects etc. from the header.py and netparms.py file, make your changes in there as needed
from netparms import *
from header_better import *
import glob
import losspad
from numpy import random as npr
#OV add matplotlib for plotting and cross checks
#import matplotlib.pyplot as plt
import random as rd
import string as string
####END IMPORTS
tf.__version__


'2.4.1'

DEFINITIONS

In [4]:
#patch generator (including noise augmentation) 
def generator(x, y, batch_size, augment_noise=.1):
    batch_inputs = np.zeros((batch_size,)+x.shape[1:4])
    batch_outputs = np.zeros((batch_size,)+y.shape[1:4])
    masks = []
    for sno in range(x.shape[0]):
        mask_ind = np.where(x[sno,...] == 0)
        masks.append(mask_ind)

    found = 0
    while (True):
        sno = npr.randint(0, x.shape[0])
        inp = x[sno,...].copy()
        inp += npr.uniform(-augment_noise, +augment_noise)
        inp[masks[sno]] = 0
        batch_inputs[found,...] = inp
        batch_outputs[found,...] = y[sno,...]
        found = found + 1
        if found >= batch_size:
            yield batch_inputs, batch_outputs
            found = 0

#Normalisation of input image (usually parameterised map)
def standardize_image(input_2d):

   #Label zeros and normalise to normal distribution   
   image_copy = np.copy(input_2d)
   label_image = label(image_copy == 0)
   largest_label, largest_area = None, 0
   
   for region in regionprops(label_image):
      if region.area > largest_area:
         largest_area = region.area
         largest_label = region.label
   
   mask = label_image == largest_label
   masked_image = np.ma.masked_where(mask, image_copy)
   
   masked_image     = masked_image - np.mean(masked_image)
   masked_image     = masked_image / np.std(masked_image)
   standardized_image = np.ma.getdata(masked_image)

   return standardized_image

#define maskes loss function 
def masked_MSE_loss(y_true, y_pred):
    mask_value   = 0
    mask_image   = tf.cast(y_true != 0, tf.float32)
    squaredError = tf.math.squared_difference(y_true,y_pred)*mask_image
    lval         = tf.reduce_mean(squaredError)
    return lval 
    
        

READ IN DATA


In [ ]:
#Surfaces, CNN in- and outputs (functional, angio data, thickness)
surfs_sphere   = []
surfs_pial     = []
surfs_inflated = []
labels_cortex  = []
maps_func      = []
maps_angles    = []
maps_thickness = []
maps_angio     = []

NumCrappyFunc  = 0

for subject in subjects:

   #Read in surfaces and sphere for paramterisation
   #t1 base directory with surfaces	
   anat_dir       = os.path.join(sdir, subject, '%s_T1_base' %(subject))
   
   #Initialise lists for functional and venographgy runs over hemispheres
   labels    = []
   funcs     = []
   angles    = []
   spheres   = []
   pials     = []
   inflated  = []
   thickness = []
   angios    = []

   for hno, hemi in enumerate(hemis):   
   #sphere coordinate system, lh and rh
      fname          = os.path.join(anat_dir,'surf', '%s.sphere.reg' %(hemi))
      sphere_surf    = fs.Surface.read(fname)
      spheres.append(sphere_surf)    #append each hemisphere
      #Pial surface
      fname          = os.path.join(anat_dir,'surf', '%s.pial' % (hemi))
      pial           = fs.Surface.read(fname)
      pials.append(pial)        #append each hemisphere
      #Inflafted surface
      fname          = os.path.join(anat_dir,'surf', '%s.inflated' % (hemi))
      infl           = fs.Surface.read(fname)
      inflated.append(infl)        #append each hemisphere
      #Thickness 
      fname          = os.path.join(anat_dir,'surf', '%s.thickness' %(hemi))
      ov             = fs.Overlay.read(fname)
      thick          = sphere_surf.parameterize(ov)
      thickness.append(thick)  #append each hemisphere)
      #Cortex label, use nibabel to read in, fs is deprecated
      fname          = os.path.join(anat_dir,'label', '%s.cortex.label.mgz' %(hemi))
      ov             = fs.Overlay.read(fname)
      label          = sphere_surf.parameterize(ov)
      labels.append(label)  #append each hemisphere)
      #Angiographic projections
      fname          = os.path.join(anat_dir,'mri', '%s.%s_AngioVeno_projfrac0.0.mgz' %(hemi,subject))
      ov             = fs.Overlay.read(fname)
      angio          = sphere_surf.parameterize(ov)
      angios.append(angio)  #append each hemisphere)

      #Initialise list of functional and orientation data that loops over runs
      func  = []
      angle = []

      ### Read in functional data ####
      for func_run in Func_runs:
         dir_name    = os.path.join(sdir, subject, func_run)
        
         #Not all func files could be registered, so had to throw some out
         fname    = os.path.join(dir_name,'%s.%s_%s_filtered_func_CoeffVar2white_projfrac0.6.mgz' %(hemi,subject,func_run))
         if os.path.exists(fname):
            print('Reading subject %s, functional run %s on hemisphere %s ' %(subject, func_run, hemi))
            ov       = fs.Overlay.read(fname)
            #2D representation of the cortex (surface parametrization where axes are 
            #longitude and colatitude), this python code generates parametrization 
            #from the spherical coordinates
            mrisp               = sphere_surf.parameterize(ov) #standardize_image(sphere_surf.parameterize(ov))       
#            test0               = mrisp
#            test00              = ov.data     
            target_shape_no_pad = mrisp.shape
            #data_wrapped        = np.pad(mrisp.data, ((pad,pad),(0,0)), 'wrap')
            #Append all surfaces, i.e. lh, rh, lr, rh,... don't make separate array dimension for hemispheres
            func.append(mrisp)
            #Read in orientation overlays
            fname          = os.path.join(dir_name,'%s.pial2%s_angles.mgz' %(hemi,func_run))
            ov             = fs.Overlay.read(fname)
            tmp            = ov.data[0:,4].copy()
            tmp[tmp>90]    = tmp[tmp>90]-180
            tmp            = np.absolute(tmp)
            mrisp          = sphere_surf.parameterize(tmp)
            angle.append(mrisp)

         else:
            NumCrappyFunc = NumCrappyFunc+1
       #end functional data

      funcs.append(func) #append each hemisphere
      angles.append(angle)

   #end hemispheres
   
   surfs_sphere.append(spheres)
   surfs_pial.append(pials)
   surfs_inflated.append(inflated)
   labels_cortex.append(labels)
   maps_func.append(funcs) 	#append each subject [subj, hemis, func_runs]
   maps_angles.append(angles) #append each subject    [subj, hemis, veno_runs]
   maps_thickness.append(thickness)#append each subject    [subj, hemis, thickness]
   maps_angio.append(angios)
#end subjects

#np.save('data_train/surfs_sphere.npy',surfs_sphere)
#np.save('data_train/surfs_pials.npy',surfs_pial)
#np.save('data_train/surfs_inflated.npy',surfs_inflated)
#np.save('data_train/labels_cortex.npy',labels_cortex)
#np.save('data_train/maps_func.npy',maps_func)
#np.save('data_train/maps_angles.npy',maps_angles)
#np.save('data_train/maps_thickness.npy',maps_thickness)
#np.save('data_train/maps_angio.npy',maps_angio)

In [5]:
surfs_sphere   = np.load('data_train/surfs_sphere.npy',allow_pickle = "true")
surfs_pials    = np.load('data_train/surfs_pials.npy',allow_pickle = "true")
surfs_inflated = np.load('data_train/surfs_inflated.npy',allow_pickle = "true")
labels_cortex  = np.load('data_train/labels_cortex.npy',allow_pickle = "true")
maps_func      = np.load('data_train/maps_func.npy',allow_pickle = "true")
maps_angles    = np.load('data_train/maps_angles.npy',allow_pickle = "true")
maps_thickness = np.load('data_train/maps_thickness.npy',allow_pickle = "true")
maps_angio     = np.load('data_train/maps_angio.npy',allow_pickle = "true")
#%whos

In [ ]:

#test=angio.copy()
#n,bins,patches = plt.hist(test.ravel(), bins = 'auto', alpha = 0.4, label = 'Normalised thickness')
labels_cortex.shape
labels_cortex = np.ones(labels_cortex.shape)


In [6]:
numInputs = 1
output_shape = (maps_func[0][0][0].shape+(1,)) 
input_shape  = (maps_func[0][0][0].shape+(numInputs,))
target_shape = input_shape
#target_shape = (maps_func[0][0][0].shape+(numInputs,))
print('target_shape is') 
print(target_shape)
print('input_shape')
print(input_shape)
K.clear_session()
model = ne.models.unet(nb_features, target_shape, nb_depth, (3,3), 1, nb_conv_per_level = nb_conv_per_level, \
                       batch_norm = -1, final_pred_activation = 'linear')


#Assign in- and output data
train_inputs  = []
train_outputs = [] 
for sno, subject in enumerate(subjects): 
   for hno, hemi in enumerate(hemis):
      for rn, func_runs in enumerate(Func_runs):
         if numInputs > 1:
            #We don't have a full set of runs for all subjects, some didn't register well
            #also multiply by cortex label 
            tmp_in = np.zeros(input_shape)
            try:
               train_outputs.append(np.multiply(maps_func[sno][hno][rn].data,labels_cortex[sno][hno]))
               tmp_in[...,0] = np.multiply(maps_thickness[sno][hno].data,labels_cortex[sno][hno])
               tmp_in[...,1] = np.multiply(maps_angio[sno][hno].data,labels_cortex[sno][hno])
               tmp_in[...,2] = np.multiply(maps_angles[sno][hno][rn].data,labels_cortex[sno][hno])
               train_inputs.append(tmp_in)
            except IndexError:
               continue
         elif numInputs == 1:
            try: 
               train_inputs.append(np.multiply(maps_func[sno][hno][rn].data,labels_cortex[sno][hno]))
               #train_outputs.append(maps_angio[sno][hno].data)
               train_outputs.append(np.multiply(maps_thickness[sno][hno].data,labels_cortex[sno][hno]))
               #train_outputs.append(maps_angles[sno][hno][rn].data) 
               #print("Single input")
            except IndexError:
               continue

target_shape is
(256, 512, 1)
input_shape
(256, 512, 1)


In [ ]:
#make directory for callbacks 
model_dir = 'model_dir'
RandomName=''.join(rd.sample(string.ascii_lowercase,5))
#RandomName='leave_out09_Thickness'# Thickness_Mean' #riple' #Orientation' #Thickness' #AngioVeno_projfrac0.0'
os.mkdir(os.path.join('/cluster/visuo/users/olivia/Projects/DeepBias/Better_code',model_dir,RandomName))
save_file_name = os.path.join(model_dir, RandomName, '{epoch:02d}.h5') 
save_callbacks = tf.keras.callbacks.ModelCheckpoint(save_file_name)
#subclass of lr_callback written by Bruce. If loss explodes it detects that and falls back to a checkpoint
#lr_callback    = nes.tf.callbacks.ReduceLRWithModelCheckpointAndRecovery(save_file_name)
  
sphereloss = losspad.spherical_loss(target_shape[0:2],pad=pad) #_no_pad[0:2],pad=pad)
#OV: neuron (ne) expects another channel, but here we just have one number (the thickness or fmri derived parameter), 
#maybe replace that with the time series itself? for now we just add np.newaxis as an empty channel
if numInputs > 1:
   x = np.array(train_inputs)
   y = np.array(train_outputs)[...,np.newaxis]
elif numInputs == 1:
   x = np.array(train_inputs)[...,np.newaxis]
   y = np.array(train_outputs)[...,np.newaxis]

print('input dimensions ' + str(x.shape))
print('output dimensions' + str(y.shape))


Total_samples = x.shape[0]
#OV train on first subject
xtrain = x[0:(Total_samples-20),...]
ytrain = y[0:(Total_samples-20),...]
xtest  = x[(Total_samples-1):Total_samples,...] #left and right hemisphere of last subject
ytest  = y[(Total_samples-1):Total_samples,...]
print('input train dimensions '  + str(xtrain.shape))
print('input test dimensionss'   + str(xtest.shape))
print('output train dimensions ' + str(ytrain.shape))
print('output test dimensionss'  + str(ytest.shape))


learning_rate = 1e-3
use_mask   = True
if use_mask == False:
    print('NOT using masked loss')
    #OV changed to tensorflow.keras bc it gave attribute error otherwise
    model.compile(optimizer=tf.keras.optimizers.Adam(lr=learning_rate), loss='MSE', metrics = ['accuracy'])
else:
    print('using masked loss')
    model.compile(optimizer=Adam(lr=learning_rate), loss=masked_MSE_loss , metrics = ['accuracy'])

#Fit
#dg = generator(xtrain[...,np.newaxis], ytrain, 8,augment_noise=.3)
dg    = generator(xtrain, ytrain, 8, augment_noise=.3)
epochs= 20
fhist = model.fit(dg, steps_per_epoch=1024, epochs=epochs, \
                  validation_data= (xtest, ytest), \
                  verbose=1, \
                  callbacks=[save_callbacks])
loss     = fhist.history['loss']
val_loss = fhist.history['val_loss']

input dimensions (162, 256, 512, 1)
output dimensions(162, 256, 512, 1)
input train dimensions (142, 256, 512, 1)
input test dimensionss(1, 256, 512, 1)
output train dimensions (142, 256, 512, 1)
output test dimensionss(1, 256, 512, 1)
using masked loss
Epoch 1/20
1024/1024 [==============================] - 275s 220ms/step - loss: 2.3703 - accuracy: 0.0256 - val_loss: 0.4555 - val_accuracy: 0.0000e+00
Epoch 2/20
1024/1024 [==============================] - 206s 201ms/step - loss: 0.3995 - accuracy: 1.7252e-06 - val_loss: 0.4604 - val_accuracy: 0.0000e+00
Epoch 3/20
1024/1024 [==============================] - 206s 201ms/step - loss: 0.3914 - accuracy: 1.0652e-06 - val_loss: 0.4895 - val_accuracy: 0.0000e+00
Epoch 4/20
1024/1024 [==============================] - 206s 201ms/step - loss: 0.3838 - accuracy: 1.6141e-06 - val_loss: 0.3867 - val_accuracy: 0.0000e+00
Epoch 5/20
1024/1024 [==============================] - 206s 202ms/step - loss: 0.3421 - accuracy: 7.0372e-06 - val_loss: 0.35

In [ ]:

ep = range(epochs)
plt.figure()
plt.plot(ep,loss, label='train loss')
plt.plot(ep,val_loss,label = 'val loss')
plt.xlabel('epochs')
plt.legend()
plt.show()
#model.save(os.path.join(model_dir, RandomName) + '/final_model.h5')
#trained_model = tf.keras.models.load_model(os.path.join(model_dir,RandomName) + '/final_model.h5'

#Test on unseen data from unseen subject
yp                 = model.predict(xtest)
#Take out non cortex labeled areas
yp[xtest==0]       = 0
#Mean differene between predicition and truth
print('Prediciting error is ' + str(np.absolute(ytest-yp).mean()))
rand               = ytest.copy().squeeze()
#Error for scrambled prediction
np.random.shuffle(rand)
print('Random error is ' + str(np.absolute(ytest.squeeze()-rand).mean()))
#Error for scrambled predictor (input)
bonkers           = xtest.copy().squeeze()
np.random.shuffle(bonkers)
bonkers           = np.array(bonkers)[np.newaxis,...,np.newaxis]
yp_bonkers        = model.predict(bonkers)
print('Error between bonkers input predictino and truth is ' + \
      str(np.absolute(yp_bonkers.squeeze() - ytest.squeeze()).mean()))



#Histogram of predicition error for orientation (not in percent)
fig = plt.figure()
tmp = ytest-yp
tmp = tmp[~np.isnan(tmp)]
tmp = tmp[~np.isinf(tmp)]
tmp2 = ytest.squeeze()-rand
tmp2 = tmp2[~np.isnan(tmp2)]
tmp2 = tmp2[~np.isinf(tmp2)] 
n,bins,patches = plt.hist(tmp, bins = 'auto', alpha = 0.4, label = 'Prediction')
plt.hist(tmp2, bins = bins, alpha = 0.4, label = 'Randomised orientation')
plt.xlabel('prediction error [\deg]')
plt.ylabel('counts')
plt.xlim(-90,90)
#plt.legend()
plt.title('Prediction error of orientation from fMRI')
plt.show()

In [ ]:
sphere      = surfs_sphere[-1][-1]
pial_disp   = surfs_inflated[-1][-1]
fv          = fs.Freeview()
truth       = sphere.sample_parameterization(ytest.squeeze())
overlay     = fv.OverlayTag(truth, name='truth')#, threshold=(.0, 90)) 
overlays    = [overlay]
prediction  = sphere.sample_parameterization(yp.squeeze())
overlay     = fv.OverlayTag(prediction, name='prediction')#, threshold=(0, 90)) 
overlays.append(overlay)
difference  = sphere.sample_parameterization(ytest.squeeze()-yp.squeeze())
overlay     = fv.OverlayTag(difference, name='difference')#, threshold=(-50,50)) 
overlays.append(overlay)


fv.surf(pial_disp, overlay=overlays)
fv.show(threads=20)

#fv.show.__code__.co_varnames

#Plot model overview
#keras.utils.plot_model(model,to_file='model.png',show_shapes=True,show_layer_names=True)

In [ ]:
np.mean(xtest)
xtest[:,:,:,0].std()